## Finetuning YOLOv11 with Buffalo and Camel labels.
For installation and setup, view the [README](./README.md) before running this.

As mentioned in the README, we will be using Roboflow for labelling our dataset.

The Roboflow project is [available here](https://app.roboflow.com/stuff-avvl2/yolo-fine-y444l).

## Download Annotated Dataset


## Training the Model

### Setup the logging

Ultralytics support logging to wandb, comet.ml and tensorboard, out of the box. Here we only enable wandb.

You need to create an account at [wandb](https://wandb.ai) and get the API key from https://wandb.ai/authorize.